In [1]:
# Import important libraries
!pip install requests bs4 selenium webdriver-manager pandas sqlalchemy psycopg2-binary ipython-sql prettytable==3.9.0

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached bs4-0.0.2-py2.py3-none-any.whl.metadata (411 bytes)
  Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl.metadata (12 kB)
  Using cached pandas-2.3.3-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached sqlalchemy-2.0.44-cp313-cp313-win_amd64.whl.metadata (9.8 kB)
  Using cached psycopg2_binary-2.9.11-cp313-cp313-win_amd64.whl.metadata (5.1 kB)
  Using cached ipython_sql-0.5.0-py3-none-any.whl.metadata (17 kB)
  Using cached prettytable-3.9.0-py3-none-any.whl.metadata (26 kB)
  Using cached charset_normalizer-3.4.4-cp313-cp313-win_amd64.whl.metadata (38 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.10.5-py3-none-any.whl.metadata (2.5 kB)
  Using cached beautifulsoup4-4.14.2-py3-none-any.whl.metadata (3.8 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using

In [ ]:
# Import required libraries
import pandas as pd
from sqlalchemy import create_engine
import requests
from bs4 import BeautifulSoup
import psycopg2


In [ ]:
# Define the target URL (IMDb Top 250 Movies page)
url = "https://www.imdb.com/chart/top/"

# Define custom headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(url, headers=headers)

# Parse the HTML content of the page using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')



In [7]:
# Create an empty list to store movie titles
movie_titles = []

# Find all title div elements using IMDb's current CSS class
titles = soup.find_all(
    "div",
    class_="ipc-title ipc-title--base ipc-title--title ipc-title--title--reduced ipc-title-link-no-icon ipc-title--on-textPrimary sc-87337ed2-2 dRlLYG cli-title with-margin"
)

# Loop through the first 20 movie titles and extract text
for t in titles[:20]:
    movie_titles.append(t.get_text(strip=True))

# Display extracted titles
print(movie_titles)


['The Shawshank Redemption', 'The Godfather', 'The Dark Knight', 'The Godfather Part II', '12 Angry Men', 'The Lord of the Rings: The Return of the King', "Schindler's List", 'The Lord of the Rings: The Fellowship of the Ring', 'Pulp Fiction', 'The Good, the Bad and the Ugly', 'The Lord of the Rings: The Two Towers', 'Forrest Gump', 'Fight Club', 'Inception', 'Star Wars: Episode V - The Empire Strikes Back', 'The Matrix', 'GoodFellas', 'Interstellar', "One Flew Over the Cuckoo's Nest", 'Seven']


In [8]:
# Create an empty list to store movie release years
movie_years = []

# Find all 'div' elements that contain the movie year
year_elements = soup.find_all(
    "div",
    class_="sc-caa65599-6 hhUutV cli-title-metadata"
)

# Loop through the first 20 elements and extract the year
for y in year_elements[:20]:
    text = y.get_text(strip=True)
    release_year = text[:4]  # Extract the first 4 characters (the year)
    movie_years.append(release_year)

# Display the extracted years
print(movie_years)


['1994', '1972', '2008', '1974', '1957', '2003', '1993', '2001', '1994', '1966', '2002', '1994', '1999', '2010', '1980', '1999', '1990', '2014', '1975', '1995']


In [9]:
# Create an empty list to store IMDb ratings
movie_ratings = []

# Find all span elements containing the movie ratings
rating_elements = soup.find_all("span", class_="ipc-rating-star--rating")

# Loop through the first 20 ratings and extract text
for r in rating_elements[:20]:
    rating_value = r.get_text(strip=True)
    movie_ratings.append(rating_value)

# Display extracted ratings
print(movie_ratings)


['9.3', '9.2', '9.1', '9.0', '9.0', '9.0', '9.0', '8.9', '8.8', '8.8', '8.8', '8.8', '8.8', '8.8', '8.7', '8.7', '8.7', '8.7', '8.6', '8.6']


In [11]:
# Combine scraped lists into a dictionary
movies_data = {
    "title": movie_titles,
    "year": movie_years,
    "rating": movie_ratings
}

# Convert the dictionary into a pandas DataFrame
movie_df = pd.DataFrame(movies_data)

# Display the DataFrame
print("IMDb Top 20 Movies:")
display(movie_df.head())



IMDb Top 20 Movies:


,title,year,rating
0,The Shawshank Redemption,1994,9.3
1,The Godfather,1972,9.2
2,The Dark Knight,2008,9.1
3,The Godfather Part II,1974,9.0
4,12 Angry Men,1957,9.0


#### Data Cleaning

In [12]:
# Convert 'movie_year' to numeric (integer)
movie_df["year"] = pd.to_numeric(movie_df["year"], errors="coerce").astype("Int64")

# Convert 'movie_rating' to float
movie_df["rating"] = pd.to_numeric(movie_df["rating"], errors="coerce").astype(float)

# Display the cleaned DataFrame
print("Cleaned and formatted:")
display(movie_df)


Cleaned and formatted:


,title,year,rating
0,The Shawshank Redemption,1994,9.3
1,The Godfather,1972,9.2
2,The Dark Knight,2008,9.1
3,The Godfather Part II,1974,9.0
4,12 Angry Men,1957,9.0
5,The Lord of the Rings: The Return of the King,2003,9.0
6,Schindler's List,1993,9.0
7,The Lord of the Rings: The Fellowship of the Ring,2001,8.9
8,Pulp Fiction,1994,8.8
9,"The Good, the Bad and the Ugly",1966,8.8


In [13]:
# Keep only the relevant columns
movie_df = movie_df[["title", "year", "rating"]]

# Create a sequential ID starting from 1
movie_df["ID"] = range(1, len(movie_df) + 1)

# Reorder columns so ID appears first
movie_df = movie_df[["ID", "title", "year", "rating"]]

# Display the updated DataFrame
print("Final cleaned:")
display(movie_df.head())



Final cleaned:


,ID,title,year,rating
0,1,The Shawshank Redemption,1994,9.3
1,2,The Godfather,1972,9.2
2,3,The Dark Knight,2008,9.1
3,4,The Godfather Part II,1974,9.0
4,5,12 Angry Men,1957,9.0


In [ ]:
# Exporting to CSV
output_path = r"cleaned_data\movies.csv"

# Save the cleaned data to a CSV file
movie_df.to_csv(output_path, index=False)

print(f"Data successfully saved to: {output_path}")


Data successfully saved to: cleaned_data\movies.csv


In [15]:
# Import SQLAlchemy library
from sqlalchemy import create_engine

# Define PostgreSQL database connection parameters 
db_name = "movies_IMDb"
db_host = "localhost"
db_user = "postgres"
db_password = "London123"
db_port = "5432"



In [16]:
# Create SQLAlchemy engine
engine = create_engine(
    f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'
)

In [17]:
# Load DataFrames to PostgreSQL table
movie_df.to_sql('movies', engine, index=False, if_exists='replace')


print("Table loaded successfully to PostgreSQL.")


Table loaded successfully to PostgreSQL.
